# **WorldPop versus DHIS2 population comparison analysis**


In [ ]:
# Set SNT Paths
SNT_ROOT_PATH  <- "~/workspace"
CODE_PATH      <- file.path(SNT_ROOT_PATH, "code")
CONFIG_PATH    <- file.path(SNT_ROOT_PATH, "configuration")
 
# load util functions
source(file.path(CODE_PATH, "snt_utils.r"))

# List required packages 
required_packages <- c("tidyr", "terra", "arrow", "sf", "dplyr", "ggplot2", "reticulate") # reticulate

# Execute function
install_and_load(required_packages)

In [ ]:
# Set variables to load openhexa.sdk from the right environment
Sys.setenv(PROJ_LIB = "/opt/conda/share/proj")
Sys.setenv(GDAL_DATA = "/opt/conda/share/gdal")
Sys.setenv(RETICULATE_PYTHON = "/opt/conda/bin/python")

# Load openhexa.sdk
print(reticulate::py_config()$python)
tryCatch({ 
    openhexa <- import("openhexa.sdk") 
},
error = function(e) {
    msg <- paste0("Error while loading openhexa.sdk", conditionMessage(e))  
    cat(msg)   
    stop(msg) 
})

In [ ]:
# Load SNT config
config_json <- tryCatch({ jsonlite::fromJSON(file.path(CONFIG_PATH, "SNT_config.json"))},
    error = function(e) {
        msg <- paste0("Error while loading configuration", conditionMessage(e))  
        cat(msg)   
        stop(msg) 
    })

# DHIS2 Dataset extract identifier
worldpop_dataset <- config_json$SNT_DATASET_IDENTIFIERS$WORLDPOP_DATASET_EXTRACT
dhis2_formatted_dataset <- config_json$SNT_DATASET_IDENTIFIERS$DHIS2_DATASET_FORMATTED
COUNTRY_CODE <- config_json$SNT_CONFIG$COUNTRY_CODE

### Load data 

In [ ]:
# Select the parquet file from the WorldPop dataset.
dataset_last_version <- openhexa$workspace$get_dataset(worldpop_dataset)$latest_version
if (is.null(dataset_last_version)) {
    stop("No version available in SNT WorldPop dataset. Process stopped.")
}

parquet_file <- NULL
files_list <- reticulate::iterate(dataset_last_version$files)
for (file in files_list) {
    if (endsWith(file$filename, ".parquet")) {
        parquet_file <- file$filename
        parquet_file <- paste0(COUNTRY_CODE, "_", substring(parquet_file, 5))  # Make sure we select the country file.
        print(paste0("Parquet file found: ", parquet_file))
    }
}

if (is.null(parquet_file)) {
    stop("No *.parquet file available in SNT WorldPop dataset. Process stopped.")
}

In [ ]:
# Load worldpop population
worldpop_population <- tryCatch({ get_latest_dataset_file_in_memory(worldpop_dataset, parquet_file) },
                  error = function(e) {
                      msg <- paste("Error while loading WorldPop population file ",parquet_file," for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("WorldPop population file ",parquet_file," loaded from dataset : ", worldpop_dataset, " dataframe dimensions: ", paste(dim(worldpop_population), collapse=", "))
log_msg(msg)

In [ ]:
# Load DHIS2 population
dhis2_population <- tryCatch({ get_latest_dataset_file_in_memory(dhis2_formatted_dataset, paste0(COUNTRY_CODE, "_population.parquet")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 population file for: " , COUNTRY_CODE, conditionMessage(e))  # log error message
                      cat(msg)
                      stop(msg)
})

msg <- paste0("DHIS2 population data loaded from dataset : ", dhis2_formatted_dataset, " dataframe dimensions: ", paste(dim(dhis2_population), collapse=", "))
log_msg(msg)

In [ ]:
# Load DHIS2 shapes data
shapes_data <- tryCatch({ get_latest_dataset_file_in_memory(dhis2_formatted_dataset, paste0(COUNTRY_CODE, "_shapes.geojson")) }, 
                  error = function(e) {
                      msg <- paste("Error while loading DHIS2 Shapes data for: " , COUNTRY_CODE, conditionMessage(e))
                      cat(msg)
                      stop(msg)
                      })
msg <- paste0("DHIS2 shapes data loaded from dataset : ", dhis2_formatted_dataset, " dataframe dimensions: ", paste(dim(shapes_data), collapse=", "))
log_msg(msg)

## FOSA geolocation quality check

This section reports the share of facilities with usable geolocation data in the DHIS2 pyramid used by the healthcare access workflow.

In [ ]:
# Load DHIS2 pyramid and compute grouped facility geolocation quality metrics
pyramid_data <- tryCatch({
    get_latest_dataset_file_in_memory(dhis2_formatted_dataset, paste0(COUNTRY_CODE, "_pyramid.parquet"))
}, error = function(e) {
    msg <- paste("Error while loading DHIS2 pyramid data for:", COUNTRY_CODE, conditionMessage(e))
    cat(msg)
    stop(msg)
})

# Restrict to facility level when LEVEL is available
fosa_level <- config_json$SNT_CONFIG$ANALYTICS_ORG_UNITS_LEVEL
if ("LEVEL" %in% names(pyramid_data)) {
    fosa_data <- pyramid_data[pyramid_data$LEVEL == fosa_level, ]
} else {
    fosa_data <- pyramid_data
}

latitude_col <- grep("^LATITUDE$", names(fosa_data), ignore.case = TRUE, value = TRUE)[1]
longitude_col <- grep("^LONGITUDE$", names(fosa_data), ignore.case = TRUE, value = TRUE)[1]

if (is.na(latitude_col) || is.na(longitude_col)) {
    stop("Could not find LATITUDE/LONGITUDE columns in DHIS2 pyramid data.")
}

# Find a facility label column
facility_name_col <- paste0("LEVEL_", fosa_level, "_NAME")
if (!(facility_name_col %in% names(fosa_data))) {
    level_name_cols <- grep("^LEVEL_[0-9]+_NAME$", names(fosa_data), value = TRUE)
    if (length(level_name_cols) == 0) {
        stop("Could not identify a facility name column (LEVEL_X_NAME) in pyramid data.")
    }
    facility_name_col <- level_name_cols[length(level_name_cols)]
}

# Prefer an explicit facility type column if available
candidate_type_cols <- c(
    "TYPE_SIMPLE", "TYPE", "FACILITY_TYPE", "STRUCTURE_TYPE", "ORG_UNIT_TYPE",
    paste0("LEVEL_", fosa_level, "_TYPE")
)
existing_type_cols <- unlist(lapply(candidate_type_cols, function(cc) {
    grep(paste0("^", cc, "$"), names(fosa_data), ignore.case = TRUE, value = TRUE)
}))
selected_type_col <- if (length(existing_type_cols) > 0) existing_type_cols[1] else NA_character_

norm_type_label <- function(x) {
    x <- as.character(x)
    x <- toupper(trimws(x))
    x[x %in% c("", "NA", "N/A", "NULL")] <- NA_character_
    x <- gsub("[[:punct:]]+", " ", x)
    x <- gsub("\\s+", " ", x)
    trimws(x)
}

# Country-specific facility type regex (from facility name when no type column)
classify_from_name_ner <- function(x) {
    x <- norm_type_label(x)
    dplyr::case_when(
        grepl("HGR|HOPITAL NATIONAL|HOPITAUX NATIONAUX", x) ~ "Hôpitaux Nationaux et HGR",
        grepl("\\bCHR\\b", x) ~ "CHR",
        grepl("\\bHD\\b", x) ~ "HD",
        grepl("\\bCSI\\b", x) ~ "CSI",
        grepl("CASE DE SANTE|\\bCS\\b|CENTRE DE SANTE|CENTRE SANTE", x) ~ "Case de Santé",
        grepl("PHARMACIE|DEPOT", x) ~ "Pharmacie/Depot",
        grepl("CABINET DE SOINS", x) ~ "Cabinet de Soins",
        grepl("CABINET MEDICAL", x) ~ "Cabinet Medical",
        grepl("\\bSS\\b|SALLE D ACCOUCHEMENT", x) ~ "SS",
        grepl("MATERNITE|SOINS", x) ~ "Maternité/Soins",
        grepl("CLINIQUE|POLYCLINIQUE", x) ~ "Clinique",
        grepl("INFIRMERIE", x) ~ "Infirmerie",
        grepl("CABINET", x) ~ "Cabinet (Autres)",
        grepl("HOPITAL|HOSPITAL", x) ~ "Hôpital",
        TRUE ~ "AUTRES"
    )
}

classify_from_name_cmr <- function(x) {
    x <- norm_type_label(x)
    dplyr::case_when(
        grepl("HGR|HOPITAL NATIONAL|HOPITAUX NATIONAUX", x) ~ "Hôpitaux Nationaux et HGR",
        grepl("\\bCHR\\b", x) ~ "CHR",
        grepl("\\bHD\\b|\\bCMA\\b|\\bCMM\\b", x) ~ "HD",
        grepl("\\bCSI\\b", x) ~ "CSI",
        grepl("CASE DE SANTE|\\bCS\\b|CENTRE DE SANTE|CENTRE SANTE|HEALTH CENTER|HEALTH CENTRE|INTEGRATED HEALTH CENTER|INTEGRATED HEALTH CENTRE|DISPENSAIRE|\\bCSP\\b|\\bPSF\\b|\\bEPC\\b", x) ~ "Case de Santé",
        grepl("PHARMACIE|DEPOT", x) ~ "Pharmacie/Depot",
        grepl("CABINET DE SOINS", x) ~ "Cabinet de Soins",
        grepl("CABINET MEDICAL", x) ~ "Cabinet Medical",
        grepl("\\bSS\\b|SALLE D ACCOUCHEMENT", x) ~ "SS",
        grepl("MATERNITE|SOINS", x) ~ "Maternité/Soins",
        grepl("CLINIQUE|POLYCLINIQUE|\\bCLINIC\\b", x) ~ "Clinique",
        grepl("INFIRMERIE", x) ~ "Infirmerie",
        grepl("CABINET", x) ~ "Cabinet (Autres)",
        grepl("HOPITAL|HOSPITAL", x) ~ "Hôpital",
        TRUE ~ "AUTRES"
    )
}

classify_from_name <- function(x) {
    if (COUNTRY_CODE == "CMR") {
        classify_from_name_cmr(x)
    } else if (COUNTRY_CODE == "NER") {
        classify_from_name_ner(x)
    } else {
        classify_from_name_ner(x)
    }
}

# Build type_simple: use explicit type column when available, fallback on facility name
if (!is.na(selected_type_col)) {
    raw_type <- norm_type_label(fosa_data[[selected_type_col]])
    n_unique_type <- length(unique(raw_type[!is.na(raw_type)]))
    if (n_unique_type >= 2) {
        type_from_type_col <- classify_from_name(raw_type)
        type_from_name_col <- classify_from_name(fosa_data[[facility_name_col]])
        type_simple <- ifelse(type_from_type_col == "AUTRES", type_from_name_col, type_from_type_col)
        log_msg(paste0("Facility type grouping uses column: ", selected_type_col, " (with name fallback)"))
    } else {
        type_simple <- classify_from_name(fosa_data[[facility_name_col]])
        log_msg("Facility type column was not informative; grouping derived from facility names.")
    }
} else {
    type_simple <- classify_from_name(fosa_data[[facility_name_col]])
    log_msg("No explicit facility type column found; grouping derived from facility names.")
}

lat_vals <- suppressWarnings(as.numeric(fosa_data[[latitude_col]]))
lon_vals <- suppressWarnings(as.numeric(fosa_data[[longitude_col]]))
idx_gps_ok <- !is.na(lat_vals) & !is.na(lon_vals)

fosa_geo_by_type <- fosa_data %>%
    mutate(
        type_simple = type_simple,
        gps_ok = idx_gps_ok
    ) %>%
    group_by(type_simple) %>%
    summarise(
        `GPS OK` = sum(gps_ok, na.rm = TRUE),
        `NO GPS` = sum(!gps_ok, na.rm = TRUE),
        total = n(),
        pct_gps = round(100 * `GPS OK` / total, 1),
        .groups = "drop"
    )

# Apply a stable display order close to the Niger-style table
display_order <- c(
    "Case de Santé",
    "CSI",
    "Pharmacie/Depot",
    "Cabinet de Soins",
    "SS",
    "Clinique",
    "Infirmerie",
    "Cabinet Medical",
    "HD",
    "Maternité/Soins",
    "Cabinet (Autres)",
    "AUTRES",
    "CHR",
    "Hôpitaux Nationaux et HGR",
    "Hôpital"
)

fosa_geo_by_type <- fosa_geo_by_type %>%
    mutate(
        display_rank = match(type_simple, display_order),
        display_rank = ifelse(is.na(display_rank), 999L, display_rank)
    ) %>%
    arrange(display_rank, type_simple) %>%
    select(-display_rank)

fosa_geo_by_type <- bind_rows(
    fosa_geo_by_type,
    summarise(
        fosa_geo_by_type,
        type_simple = "TOTAL",
        `GPS OK` = sum(`GPS OK`),
        `NO GPS` = sum(`NO GPS`),
        total = sum(total),
        pct_gps = round(100 * sum(`GPS OK`) / sum(total), 1)
    )
)

fosa_geo_by_type_display <- fosa_geo_by_type %>%
    mutate(`% GPS` = paste0(format(round(pct_gps, 1), nsmall = 1), "%")) %>%
    select(type_simple, `GPS OK`, `NO GPS`, total, `% GPS`)

knitr::kable(
    fosa_geo_by_type_display,
    align = c("l", "r", "r", "r", "r"),
    caption = "Complétude GPS des formations sanitaires par catégorie"
)

## Zone de sante comparison

In [ ]:
# select the closest year (Worldpop does not update data since 2020)
worldpop_year <- min(worldpop_population$YEAR)  # min year
if (worldpop_year %in% unique(dhis2_population$YEAR)){
    dhis2_year <- worldpop_year
} else if(worldpop_year < min(dhis2_population$YEAR)){
    dhis2_year <- min(dhis2_population$YEAR)
} else {
    dhis2_year <- max(dhis2_population$YEAR)
}

print(paste0("Comparison years DHIS2: ",dhis2_year, " Worldpop : ", worldpop_year))

In [ ]:
# Select DHIS2 data to closest year
dhis2_pop_renamed <- dhis2_population %>% 
    filter(YEAR == dhis2_year) %>%      
    select(ADM2_ID, dhis2_value = POPULATION)

worldpop_pop_renamed <- worldpop_population %>% 
    filter(YEAR == worldpop_year) %>%      
    select(ADM2_ID, worldpop_value = POPULATION)

# 5. Compare WorldPop vs DHIS2 (if you have a matching ID column)
comparison_df <- left_join(shapes_data, dhis2_pop_renamed[, c("ADM2_ID", "dhis2_value")], by = "ADM2_ID")
comparison_df <- left_join(comparison_df, worldpop_pop_renamed[, c("ADM2_ID", "worldpop_value")], by = "ADM2_ID")

In [ ]:
ggplot(comparison_df, aes(x = dhis2_value, y = worldpop_value)) +
  geom_point() +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "gray") +
  labs(x = "DHIS2 Population", 
       y = "WorldPop Population", 
       title = "Comparison per ADM2") +
  theme_minimal()

In [ ]:
pop_correlation <- cor(comparison_df$dhis2_value, comparison_df$worldpop_value, method = 'pearson')
print(paste0("Correlation : ", round(pop_correlation, 2)))

In [ ]:
ggplot(comparison_df) +
  geom_sf(aes(fill = worldpop_value - dhis2_value)) +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "Difference: WorldPop - DHIS2 Population (ADM2)",
       fill = "Pop. Diff") +
  theme_minimal()

In [ ]:
sapply(
    setNames(
        list(comparison_df$dhis2_value, comparison_df$worldpop_value),
        c("DHIS2_population", "WPOP_population")
    ),
    summary
)

The above table shows that some of the values of WPOP tend to be less plausible, with some Zones de Santé (ZS) having 0 inhabitants, while the largest ZS, which is likely one of the districts of Kinshasa, appears with nearly 22 million inhabitants, thus more than the total population of Kinshasa (17 million). 

In [ ]:
comp.pop <- comparison_df %>%
  select(ADM2_ID, worldpop_value, dhis2_value) %>%
  mutate(
      diff = worldpop_value - dhis2_value,
      ratio = worldpop_value / dhis2_value,
      relative_diff = diff / dhis2_value
  )

In [ ]:
hist_pop <- st_drop_geometry(comp.pop) %>%
    select(ADM2_ID, worldpop_value, dhis2_value) %>%
    rename(
        WPOP = worldpop_value,
        DHIS2 = dhis2_value
    ) %>%
    pivot_longer(
    cols = c(WPOP, DHIS2),
    names_to = "source",
    values_to = "population"
  )

In [ ]:
ggplot(hist_pop, aes(x=population, color=source)) +
  geom_histogram(fill = NA, alpha = 0.5, position = "identity", binwidth = 10000) +
  theme_minimal()

WorldPop data appears to have more variability and some extreme outliers to the right.

In [ ]:
ggplot(comp.pop, aes(y=relative_diff)) + 
  geom_boxplot() + coord_flip() + theme_minimal()

The above plot shows that for 75% of Zones de Santé (ZS), the difference between DHIS2 and WorldPop data is within 50% of the total DHIS2, which indicates significant deviation between the two sources. Most of the remaining ZS (except the outliers represented as dots) have differences of population that are 0.5-1.5 of the total DHIS2 population.

In [ ]:
ggplot(comparison_df) +
  geom_sf(aes(fill = (worldpop_value - dhis2_value)/dhis2_value)) +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "ADM2 Relative difference: WorldPop - DHIS2 Population (reference)",
       fill = "Relative Pop. Diff") +
  theme_minimal()

## Province comparison

In [ ]:
# Group by province (ADM1) and dissolve geometries
dhis2_shapes_provinces <- shapes_data %>%
  group_by(ADM1_ID) %>%
  summarise(geometry = st_union(geometry), .groups = "drop")

# Group pop by provinces
dhis2_pop_prov <- dhis2_population %>%
  group_by(ADM1_NAME, ADM1_ID) %>%
  summarise(dhis2_value = sum(POPULATION, na.rm = TRUE))

# Group pop by provinces
worldpop_pop_prov <- worldpop_population %>%
  group_by(ADM1_NAME, ADM1_ID) %>%
  summarise(worldpop_value = sum(POPULATION, na.rm = TRUE))

comparison_df_prov <- left_join(dhis2_shapes_provinces, dhis2_pop_prov, by = c("ADM1_ID"))
comparison_df_prov <- left_join(comparison_df_prov, worldpop_pop_prov, by = c("ADM1_ID"))

In [ ]:
ggplot(comparison_df_prov, aes(x = dhis2_value, y = worldpop_value)) +
  geom_point() +
  geom_abline(slope = 1, intercept = 0, linetype = "dashed", color = "gray") +
  labs(x = "DHIS2 Population", y = "WorldPop Population", 
       title = "Comparison per ADM2") +
  theme_minimal()

In [ ]:
pop_correlation_prov <- cor(comparison_df_prov$dhis2_value, comparison_df_prov$worldpop_value, method = 'pearson')
print(paste0("Correlation : ", round(pop_correlation_prov, 2)))

In [ ]:
ggplot(comparison_df_prov) +
  geom_sf(aes(fill = worldpop_value - dhis2_value)) +
  scale_fill_gradient2(low = "blue", mid = "white", high = "red", midpoint = 0) +
  labs(title = "Difference: WorldPop - DHIS2 Population (ADM1)",
       fill = "Pop. Diff") +
  theme_minimal()

In [ ]:
sapply(
    setNames(
        list(comparison_df_prov$dhis2_value, comparison_df_prov$worldpop_value),
        c("DHIS2_population", "WPOP_population")
    ),
    summary
)